## 1. Generate Synthetic Dataset
- 1000 samples (X) with 10 time steps
- 5 features per step, filled with random values
- each sample has a binary label (y) for classification (0 or 1)

In [ ]:
import numpy as np

# Generate synthetic sequential data
def generate_sequential_data(num_samples, sequence_length, num_features):
    X = np.random.rand(num_samples, sequence_length, num_features)  # Random features
    y = np.random.randint(0, 2, num_samples)  # Binary classification labels
    return X, y

# Parameters
num_samples = 1000
sequence_length = 10
num_features = 5

# Generate dataset
X, y = generate_sequential_data(num_samples, sequence_length, num_features)

## 2. Implement RNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Define RNN model
def build_rnn_model(input_shape):
    model = Sequential([
        SimpleRNN(64, input_shape=input_shape, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build model
input_shape = (sequence_length, num_features)
model = build_rnn_model(input_shape)

## 3. Apply SISA Framwork

### 3.1 Shard the Dataset

In [ ]:
num_shards = 5
shard_size = num_samples // num_shards

# Split dataset into shards
shards_X = [X[i * shard_size:(i + 1) * shard_size] for i in range(num_shards)]
shards_y = [y[i * shard_size:(i + 1) * shard_size] for i in range(num_shards)]

### 3.2 Train Models on Each Shard

In [ ]:
models = []
for i in range(num_shards):
    print(f"Training model on shard {i+1}")
    model = build_rnn_model(input_shape)
    model.fit(shards_X[i], shards_y[i], epochs=10, batch_size=32, verbose=0)
    models.append(model)

Training model on shard 1
Training model on shard 2
Training model on shard 3
Training model on shard 4
Training model on shard 5


## 4. Evaluate Model Before Unlearning

In [ ]:
# Generate test data
X_test, y_test = generate_sequential_data(100, sequence_length, num_features)

# Aggregate predictions from all shard models
def aggregate_predictions(models, X_test):
    predictions = np.zeros((X_test.shape[0], 1))
    for model in models:
        predictions += model.predict(X_test)
    return (predictions / len(models)) > 0.5  # Binary classification threshold

# Evaluate accuracy
y_pred_before = aggregate_predictions(models, X_test)
accuracy_before = np.mean(y_pred_before.flatten() == y_test)
print(f"Accuracy before unlearning: {accuracy_before * 100:.2f}%")


1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Accuracy before unlearning: 52.00%


## 5. Perform Machine Unlearning

### 5.1 Identify the Shard

In [ ]:
data_point_index = 42  # Example index of data point to unlearn
shard_index = data_point_index // shard_size

### 5.2 Retrain the Shard Model

In [ ]:
# Exclude the data point
new_shard_X = np.delete(shards_X[shard_index], data_point_index % shard_size, axis=0)
new_shard_y = np.delete(shards_y[shard_index], data_point_index % shard_size, axis=0)

# Retrain the model
models[shard_index] = build_rnn_model(input_shape)
models[shard_index].fit(new_shard_X, new_shard_y, epochs=10, batch_size=32, verbose=0)

## 6. Evaluate Model After Unlearning

In [ ]:
# Evaluate accuracy after unlearning
y_pred_after = aggregate_predictions(models, X_test)
accuracy_after = np.mean(y_pred_after.flatten() == y_test)
print(f"Accuracy after unlearning: {accuracy_after * 100:.2f}%")


1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 912us/step

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step
Accuracy after unlearning: 56.00%


## Ogay Done

In [ ]:
print(f"Accuracy before unlearning: {accuracy_before * 100:.2f}%")
print(f"Accuracy after unlearning: {accuracy_after * 100:.2f}%")

Accuracy before unlearning: 52.00%
Accuracy after unlearning: 56.00%
